In [1]:
pip install python-chess


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 7.6 MB/s eta 0:00:0000:01:00:01
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'chess' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'chess'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147878 sha256=e1d46694a747bebd09e5f2245ad0cbf185e2b3affb35106c7beefa0ee1bbeb1a
  Stored in directory: /Users/prithuljoshi/Library/Caches/pip/wheels/74/65/99/8c3a88ba852c1ed8d34daed144b8cb4c654d638677483d19ff
Successfully built chess
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [python-chess]

[notice] A new release of pip is available: 25.1

In [1]:
"""
chess_ai.py
A single-file Human vs Computer Chess app using Tkinter + python-chess.
Usage:
    pip install python-chess
    python chess_ai.py
Controls:
 - Click a square to select a piece, then click destination to move.
 - After your move (White by default), the AI (Black) will compute its move.
 - Use the "New Game" button to reset. Difficulty controls search depth.
"""

import tkinter as tk
from tkinter import messagebox
import chess
import threading
import time

# -------------------------
# Configuration
# -------------------------
BOARD_SIZE = 8
SQUARE_SIZE = 64
WHITE_BG = "#F0D9B5"
BLACK_BG = "#B58863"
HIGHLIGHT_BG = "#f7ec8e"
AI_THINKING_MSG = "AI thinking..."
DEFAULT_SEARCH_DEPTH = 3  # change for difficulty (3-4 is ok for quick play)

# Unicode pieces
UNICODE_PIECES = {
    chess.PAWN:   ('♙', '♟'),
    chess.ROOK:   ('♖', '♜'),
    chess.KNIGHT: ('♘', '♞'),
    chess.BISHOP: ('♗', '♝'),
    chess.QUEEN:  ('♕', '♛'),
    chess.KING:   ('♔', '♚'),
}

# Piece-square tables (simple positional bonuses) — smaller, simple tables
# For brevity and speed these are symmetric/simple; used by eval
PST = {
    chess.PAWN: [
         0,  5,  5,  0,  5, 10, 50,  0,
         0, 10,-10,  0, 10, 30, 50,  0,
         0, 10,-10,  0, 10, 30, 50,  0,
         0,  0,  0, 20, 25, 30, 50,  0,
         0,  5,  5, 25, 30, 35, 50,  0,
         0,  5,  5, 15, 20, 25, 50,  0,
         0,  0,  0, -5, -5,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0
    ],
    chess.KNIGHT: [ -50,-40,-30,-30,-30,-30,-40,-50,
                    -40,-20,  0,  5,  5,  0,-20,-40,
                    -30,  5, 10, 15, 15, 10,  5,-30,
                    -30,  0, 15, 20, 20, 15,  0,-30,
                    -30,  5, 15, 20, 20, 15,  5,-30,
                    -30,  0, 10, 15, 15, 10,  0,-30,
                    -40,-20,  0,  0,  0,  0,-20,-40,
                    -50,-40,-30,-30,-30,-30,-40,-50],
    chess.BISHOP: [ -20,-10,-10,-10,-10,-10,-10,-20,
                    -10,  0,  0,  0,  0,  0,  0,-10,
                    -10,  0,  5, 10, 10,  5,  0,-10,
                    -10,  5,  5, 10, 10,  5,  5,-10,
                    -10,  0, 10, 10, 10, 10,  0,-10,
                    -10, 10, 10, 10, 10, 10, 10,-10,
                    -10,  5,  0,  0,  0,  0,  5,-10,
                    -20,-10,-10,-10,-10,-10,-10,-20],
    chess.ROOK:   [  0,  0,  5, 10, 10,  5,  0,  0,
                     0,  0,  5, 10, 10,  5,  0,  0,
                     0,  0,  5, 10, 10,  5,  0,  0,
                     0,  0,  5, 10, 10,  5,  0,  0,
                     0,  0,  5, 10, 10,  5,  0,  0,
                     0,  0,  5, 10, 10,  5,  0,  0,
                    25, 25, 25, 25, 25, 25, 25, 25,
                     0,  0,  5, 10, 10,  5,  0,  0],
    chess.QUEEN:  [ -20,-10,-10, -5, -5,-10,-10,-20,
                    -10,  0,  0,  0,  0,  0,  0,-10,
                    -10,  0,  5,  5,  5,  5,  0,-10,
                     -5,  0,  5,  5,  5,  5,  0, -5,
                      0,  0,  5,  5,  5,  5,  0, -5,
                    -10,  5,  5,  5,  5,  5,  0,-10,
                    -10,  0,  5,  0,  0,  0,  0,-10,
                    -20,-10,-10, -5, -5,-10,-10,-20],
    chess.KING:   [ -30,-40,-40,-50,-50,-40,-40,-30,
                    -30,-40,-40,-50,-50,-40,-40,-30,
                    -30,-40,-40,-50,-50,-40,-40,-30,
                    -30,-40,-40,-50,-50,-40,-40,-30,
                    -20,-30,-30,-40,-40,-30,-30,-20,
                    -10,-20,-20,-20,-20,-20,-20,-10,
                     20, 20,  0,  0,  0,  0, 20, 20,
                     20, 30, 10,  0,  0, 10, 30, 20],
}

# -------------------------
# Engine: evaluation + minimax with alpha-beta
# -------------------------
def evaluate_board(board: chess.Board) -> int:
    """
    Positive => advantage for White, negative => advantage for Black
    Simple material + piece-square evaluation.
    """
    if board.is_checkmate():
        # If side to move is checkmated => bad for that side
        return -99999 if board.turn else 99999
    if board.is_stalemate() or board.is_insufficient_material():
        return 0

    material = 0
    pst_score = 0

    for square in chess.SQUARES:
        piece = board.piece_at(square)
        if piece is None:
            continue
        val = 0
        if piece.piece_type == chess.PAWN:
            val = 100
        elif piece.piece_type == chess.KNIGHT:
            val = 320
        elif piece.piece_type == chess.BISHOP:
            val = 330
        elif piece.piece_type == chess.ROOK:
            val = 500
        elif piece.piece_type == chess.QUEEN:
            val = 900
        elif piece.piece_type == chess.KING:
            val = 20000

        # piece color orientation for PST: use MIRROR for black
        pst_list = PST.get(piece.piece_type, [0]*64)
        idx = square if piece.color == chess.WHITE else chess.square_mirror(square)
        pst_val = pst_list[idx]

        if piece.color == chess.WHITE:
            material += val
            pst_score += pst_val
        else:
            material -= val
            pst_score -= pst_val

    # Mobility bonus (number of legal moves)
    mobility = len(list(board.legal_moves))
    # For side to move: small bonus to encourage initiative
    mobility_score = mobility if board.turn else -mobility

    total = material + pst_score + mobility_score*5
    return total

def minimax_root(board: chess.Board, depth: int, is_maximizing: bool):
    best_move = None
    best_value = -999999 if is_maximizing else 999999

    moves = list(board.legal_moves)
    # Move ordering: try captures first heuristically
    moves.sort(key=lambda m: 0 if board.is_capture(m) else 1)

    for move in moves:
        board.push(move)
        val = minimax(board, depth-1, -999999, 999999, not is_maximizing)
        board.pop()
        if is_maximizing:
            if val > best_value:
                best_value = val
                best_move = move
        else:
            if val < best_value:
                best_value = val
                best_move = move
    return best_move, best_value

def minimax(board: chess.Board, depth: int, alpha: int, beta: int, is_maximizing: bool) -> int:
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)

    if is_maximizing:
        max_eval = -999999
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth-1, alpha, beta, False)
            board.pop()
            if eval > max_eval:
                max_eval = eval
            if eval > alpha:
                alpha = eval
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = 999999
        for move in board.legal_moves:
            board.push(move)
            eval = minimax(board, depth-1, alpha, beta, True)
            board.pop()
            if eval < min_eval:
                min_eval = eval
            if eval < beta:
                beta = eval
            if beta <= alpha:
                break
        return min_eval

# -------------------------
# GUI: Tkinter chessboard
# -------------------------
class ChessGUI:
    def __init__(self, master):
        self.master = master
        master.title("Chess — Human (White) vs AI (Black)")
        self.board = chess.Board()
        self.selected_square = None
        self.buttons = {}
        self.search_depth = tk.IntVar(value=DEFAULT_SEARCH_DEPTH)

        top_frame = tk.Frame(master)
        top_frame.pack(side=tk.TOP, pady=6)

        self.status_label = tk.Label(top_frame, text="White to move", font=("Helvetica", 12))
        self.status_label.pack(side=tk.LEFT, padx=10)

        tk.Button(top_frame, text="New Game", command=self.new_game).pack(side=tk.LEFT, padx=6)
        tk.Label(top_frame, text="AI Depth:").pack(side=tk.LEFT, padx=(10,0))
        depth_spin = tk.Spinbox(top_frame, from_=1, to=4, width=3, textvariable=self.search_depth)
        depth_spin.pack(side=tk.LEFT)

        # Board frame
        board_frame = tk.Frame(master)
        board_frame.pack()

        for r in range(8):
            for c in range(8):
                sq = chess.square(c, 7-r)  # chess.square(file, rank)
                color = WHITE_BG if (r + c) % 2 == 0 else BLACK_BG
                btn = tk.Button(board_frame,
                                text="",
                                font=("Helvetica", 32),
                                width=2,
                                height=1,
                                bg=color,
                                activebackground=HIGHLIGHT_BG,
                                command=lambda s=sq: self.on_square_click(s))
                btn.grid(row=r, column=c)
                self.buttons[sq] = btn

        self.update_board()

    def new_game(self):
        self.board.reset()
        self.selected_square = None
        self.update_board()
        self.status_label.config(text="White to move")

    def on_square_click(self, square):
        # If game over => do nothing
        if self.board.is_game_over():
            messagebox.showinfo("Game Over", f"Game over: {self.board.result()}")
            return

        piece = self.board.piece_at(square)
        if self.selected_square is None:
            # pick up only your pieces (White)
            if piece is None or piece.color != chess.WHITE:
                return
            self.selected_square = square
            self.highlight_legal_moves(square)
        else:
            # attempt move from selected_square to clicked square
            move = chess.Move(self.selected_square, square)
            # handle promotions: if pawn reaches last rank, default to queen for simplicity
            if self.board.piece_at(self.selected_square).piece_type == chess.PAWN and chess.square_rank(square) in (0,7):
                move = chess.Move(self.selected_square, square, promotion=chess.QUEEN)

            if move in self.board.legal_moves:
                self.board.push(move)
                self.selected_square = None
                self.update_board()
                self.status_label.config(text="AI thinking...")
                self.master.update()
                # After human move, trigger AI move synchronously (quick)
                # For better UX you could run AI in a thread and disable UI
                self.master.after(100, lambda: self.ai_move())
            else:
                # If click on another white piece, change selection
                if piece is not None and piece.color == chess.WHITE:
                    self.selected_square = square
                    self.highlight_legal_moves(square)
                else:
                    # clear selection
                    self.selected_square = None
                    self.update_board()

    def highlight_legal_moves(self, square):
        self.update_board()  # clear highlights first
        btn = self.buttons[square]
        btn.config(bg=HIGHLIGHT_BG)
        for move in self.board.legal_moves:
            if move.from_square == square:
                dest_btn = self.buttons[move.to_square]
                dest_btn.config(bg="#a2f5a2")  # highlight possible moves

    def ai_move(self):
        if self.board.is_game_over():
            self.update_board()
            self.show_game_over()
            return
        depth = max(1, int(self.search_depth.get()))
        # We are assuming AI plays Black. If it's Black to move:
        if not self.board.turn:
            start = time.time()
            move, val = minimax_root(self.board, depth, is_maximizing=False)
            end = time.time()
            if move is None:
                # no legal move
                self.update_board()
                self.show_game_over()
                return
            self.board.push(move)
            self.update_board()
            self.status_label.config(text=f"Your move — AI eval {val} (took {end-start:.2f}s)")
            if self.board.is_game_over():
                self.show_game_over()
        else:
            # if somehow it's still white to move, just return
            self.update_board()

    def show_game_over(self):
        res = "Game over.\n"
        if self.board.is_checkmate():
            winner = "Black (AI)" if self.board.turn == chess.WHITE else "White (You)"
            res += f"Checkmate. Winner: {winner}\n"
        elif self.board.is_stalemate():
            res += "Stalemate.\n"
        elif self.board.is_insufficient_material():
            res += "Draw — insufficient material.\n"
        elif self.board.can_claim_threefold_repetition():
            res += "Draw — threefold repetition possible.\n"
        res += f"Result: {self.board.result()}"
        messagebox.showinfo("Game Over", res)
        self.status_label.config(text=res)

    def update_board(self):
        # Update button texts and restore colors
        for square, btn in self.buttons.items():
            piece = self.board.piece_at(square)
            if piece is None:
                btn.config(text="")
            else:
                glyph = UNICODE_PIECES[piece.piece_type][0] if piece.color == chess.WHITE else UNICODE_PIECES[piece.piece_type][1]
                btn.config(text=glyph)
            # reset color
            rank = chess.square_rank(square)
            file = chess.square_file(square)
            r = 7 - rank
            c = file
            color = WHITE_BG if (r + c) % 2 == 0 else BLACK_BG
            btn.config(bg=color)

        # If it's AI turn (Black), trigger AI move automatically
        if not self.board.turn and not self.board.is_game_over():
            # Very short delay to allow GUI to refresh
            self.master.after(200, lambda: self.ai_move())

# -------------------------
# Run
# -------------------------
def main():
    root = tk.Tk()
    app = ChessGUI(root)
    root.resizable(False, False)
    root.mainloop()

if __name__ == "__main__":
    main()


: 

In [ ]:
"""
upgraded_chess_gui.py
Single-file Human vs Computer Chess app (Tkinter + python-chess).

Features:
 - Canvas-based graphical board with colored piece "plaques" (no external images).
 - Last-move highlight, legal-move dots, hover selection visuals.
 - Move list, New Game, Undo, AI depth slider.
 - AI uses minimax with alpha-beta (simple evaluation). AI runs in a thread so the GUI stays responsive.
 - You play White by default; AI is Black.

Dependencies:
    pip install python-chess

Run:
    python upgraded_chess_gui.py
"""

import tkinter as tk
from tkinter import ttk, messagebox
import chess
import threading
import time

# -------------------------
# Configuration / Style
# -------------------------
SQUARE_SIZE = 80
BOARD_PIXEL = 8 * SQUARE_SIZE
PADDING = 10
BOARD_BG = "#e6d9c6"    # outside board background
LIGHT_SQUARE = "#F0D9B5"  # light
DARK_SQUARE = "#B58863"   # dark
HIGHLIGHT_COLOR = "#f7ec8e"
LAST_MOVE_COLOR = "#bfe28f"
MOVE_DOT_COLOR = "#1f7a1f"
SELECT_RING = "#ffd54f"
PIECE_FONT = ("Segoe UI Symbol", 38, "bold")  # good glyph support on many systems
STATUS_FONT = ("Helvetica", 12)
MOVE_LIST_FONT = ("Consolas", 10)
DEFAULT_AI_DEPTH = 3
AI_THINK_TEXT = "AI is thinking..."

# Unicode chess glyphs mapping via python-chess piece symbols
UNICODE_PIECES = {
    'P': '♙', 'N': '♘', 'B': '♗', 'R': '♖', 'Q': '♕', 'K': '♔',
    'p': '♟', 'n': '♞', 'b': '♝', 'r': '♜', 'q': '♛', 'k': '♚'
}

# -------------------------
# Engine: simple eval + minimax alpha-beta
# -------------------------
def evaluate_board(board: chess.Board) -> int:
    """Simple material + mobility evaluation. Positive => White advantage."""
    if board.is_checkmate():
        return -100000 if board.turn else 100000
    if board.is_stalemate() or board.is_insufficient_material():
        return 0

    values = {chess.PAWN: 100, chess.KNIGHT: 320, chess.BISHOP: 330,
              chess.ROOK: 500, chess.QUEEN: 900, chess.KING: 20000}
    material = 0
    for sq in chess.SQUARES:
        p = board.piece_at(sq)
        if p:
            v = values.get(p.piece_type, 0)
            material += v if p.color == chess.WHITE else -v

    mobility = len(list(board.legal_moves)) * (5 if board.turn else -5)
    return material + mobility

def minimax(board: chess.Board, depth: int, alpha: int, beta: int, maximizing: bool) -> int:
    if depth == 0 or board.is_game_over():
        return evaluate_board(board)

    if maximizing:
        max_eval = -10**9
        for move in board.legal_moves:
            board.push(move)
            val = minimax(board, depth-1, alpha, beta, False)
            board.pop()
            if val > max_eval:
                max_eval = val
            if val > alpha:
                alpha = val
            if beta <= alpha:
                break
        return max_eval
    else:
        min_eval = 10**9
        for move in board.legal_moves:
            board.push(move)
            val = minimax(board, depth-1, alpha, beta, True)
            board.pop()
            if val < min_eval:
                min_eval = val
            if val < beta:
                beta = val
            if beta <= alpha:
                break
        return min_eval

def minimax_root(board: chess.Board, depth: int, maximizing: bool):
    """Return best move and its evaluation value."""
    best_move = None
    best_val = -10**9 if maximizing else 10**9
    moves = list(board.legal_moves)
    moves.sort(key=lambda m: 0 if board.is_capture(m) else 1)
    for move in moves:
        board.push(move)
        val = minimax(board, depth-1, -10**9, 10**9, not maximizing)
        board.pop()
        if maximizing and val > best_val:
            best_val = val
            best_move = move
        if not maximizing and val < best_val:
            best_val = val
            best_move = move
    return best_move, best_val

# -------------------------
# GUI: Canvas board + controls
# -------------------------
class UpgradedChessGUI:
    def __init__(self, root):
        self.root = root
        root.title("Upgraded Chess — Human vs AI")
        self.board = chess.Board()
        self.history = []
        self.selected_square = None
        self.legal_destinations = []
        self.last_move = None
        self.ai_thread = None
        self.ai_search_depth = tk.IntVar(value=DEFAULT_AI_DEPTH)
        self.ai_thinking = False

        # layout frames
        main_frame = tk.Frame(root, bg=BOARD_BG, padx=PADDING, pady=PADDING)
        main_frame.pack(fill=tk.BOTH, expand=True)

        board_frame = tk.Frame(main_frame, bg=BOARD_BG)
        board_frame.pack(side=tk.LEFT)

        right_panel = tk.Frame(main_frame, bg=BOARD_BG)
        right_panel.pack(side=tk.LEFT, padx=(12,0), fill=tk.Y)

        # Canvas board
        self.canvas = tk.Canvas(board_frame, width=BOARD_PIXEL, height=BOARD_PIXEL, bg=BOARD_BG, highlightthickness=0)
        self.canvas.pack()
        self.canvas.bind("<Button-1>", self.on_click)
        self.canvas.bind("<Motion>", self.on_motion)

        # Controls
        tk.Label(right_panel, text="Controls", font=("Helvetica", 12, "bold"), bg=BOARD_BG).pack(anchor="nw")
        ttk.Button(right_panel, text="New Game", command=self.new_game).pack(fill=tk.X, pady=(6,4))
        ttk.Button(right_panel, text="Undo", command=self.undo_move).pack(fill=tk.X, pady=(0,8))

        depth_frame = tk.Frame(right_panel, bg=BOARD_BG)
        depth_frame.pack(fill=tk.X, pady=(4,8))
        tk.Label(depth_frame, text="AI Depth:", bg=BOARD_BG).pack(side=tk.LEFT)
        ttk.Scale(depth_frame, from_=1, to=4, orient=tk.HORIZONTAL, variable=self.ai_search_depth).pack(
            side=tk.LEFT, fill=tk.X, expand=True, padx=(8,0))

        # Status label
        self.status_var = tk.StringVar(value="White to move")
        self.status_label = tk.Label(right_panel, textvariable=self.status_var, font=STATUS_FONT, bg=BOARD_BG)
        self.status_label.pack(anchor="nw", pady=(6,8))

        # Move list
        tk.Label(right_panel, text="Move List", bg=BOARD_BG).pack(anchor="nw")
        self.moves_text = tk.Text(right_panel, width=28, height=18, font=MOVE_LIST_FONT, state=tk.DISABLED, wrap=tk.WORD)
        self.moves_text.pack(pady=(4,0))
        self.moves_scroll = ttk.Scrollbar(right_panel, command=self.moves_text.yview)
        self.moves_text['yscrollcommand'] = self.moves_scroll.set
        self.moves_scroll.pack(fill=tk.Y, side=tk.RIGHT, padx=(0,4))

        # Clocks
        clocks_frame = tk.Frame(right_panel, bg=BOARD_BG)
        clocks_frame.pack(fill=tk.X, pady=(8,0))
        self.white_clock = tk.Label(clocks_frame, text="You: 00:00", bg=BOARD_BG)
        self.white_clock.pack(anchor="w")
        self.black_clock = tk.Label(clocks_frame, text="AI: 00:00", bg=BOARD_BG)
        self.black_clock.pack(anchor="w")
        self.white_time = 0.0
        self.black_time = 0.0
        self._running_clock = True
        self._start_clock_thread()

        # initial draw
        self.draw_board()
        if not self.board.turn:
            self._start_ai_move()

    # -------------------------
    # Drawing helper functions
    # -------------------------
    def draw_board(self):
        self.canvas.delete("all")
        # draw squares
        for rank in range(8):
            for file in range(8):
                x1 = file * SQUARE_SIZE
                y1 = rank * SQUARE_SIZE
                x2 = x1 + SQUARE_SIZE
                y2 = y1 + SQUARE_SIZE
                square_index = chess.square(file, 7 - rank)
                is_light = ((rank + file) % 2 == 0)
                color = LIGHT_SQUARE if is_light else DARK_SQUARE
                if self.last_move:
                    if square_index == self.last_move.from_square or square_index == self.last_move.to_square:
                        color = LAST_MOVE_COLOR
                self.canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline=color)

        # highlight selected square
        if self.selected_square is not None:
            r = 7 - chess.square_rank(self.selected_square)
            c = chess.square_file(self.selected_square)
            x1, y1 = c * SQUARE_SIZE, r * SQUARE_SIZE
            x2, y2 = x1 + SQUARE_SIZE, y1 + SQUARE_SIZE
            self.canvas.create_rectangle(x1+4, y1+4, x2-4, y2-4, outline=SELECT_RING, width=4)

        # draw legal move dots
        for dest in self.legal_destinations:
            r = 7 - chess.square_rank(dest)
            c = chess.square_file(dest)
            cx, cy = c * SQUARE_SIZE + SQUARE_SIZE/2, r * SQUARE_SIZE + SQUARE_SIZE/2
            rdot = SQUARE_SIZE * 0.09
            self.canvas.create_oval(cx-rdot, cy-rdot, cx+rdot, cy+rdot, fill=MOVE_DOT_COLOR, outline="")

        # draw pieces as circular plaques + glyphs
        for sq in chess.SQUARES:
            piece = self.board.piece_at(sq)
            if not piece:
                continue
            file = chess.square_file(sq)
            rank = chess.square_rank(sq)
            canvas_r, canvas_c = 7 - rank, file
            cx = canvas_c * SQUARE_SIZE + SQUARE_SIZE/2
            cy = canvas_r * SQUARE_SIZE + SQUARE_SIZE/2
            plaque_radius = SQUARE_SIZE * 0.38
            plaque_fill = "#ffffff" if piece.color == chess.WHITE else "#2b2b2b"
            plaque_outline = "#e8e8e8" if piece.color == chess.WHITE else "#111111"
            # shadow for depth (fixed: no alpha hex)
            self.canvas.create_oval(cx-plaque_radius+2, cy-plaque_radius+2,
                                    cx+plaque_radius+2, cy+plaque_radius+2,
                                    fill="#000000", outline="", stipple="gray25", tags="shadow")
            self.canvas.create_oval(cx-plaque_radius, cy-plaque_radius,
                                    cx+plaque_radius, cy+plaque_radius,
                                    fill=plaque_fill, outline=plaque_outline, width=2)
            glyph = UNICODE_PIECES[piece.symbol()]
            text_color = "#000000" if piece.color == chess.WHITE else "#ffffff"
            self.canvas.create_text(cx, cy, text=glyph, fill=text_color, font=PIECE_FONT)

    # -------------------------
    # Interaction handlers
    # -------------------------
    def on_click(self, event):
        if self.ai_thinking:
            return
        file, rank = event.x // SQUARE_SIZE, event.y // SQUARE_SIZE
        if not (0 <= file < 8 and 0 <= rank < 8):
            return
        square = chess.square(file, 7 - rank)
        if self.selected_square is None:
            piece = self.board.piece_at(square)
            if piece is None or piece.color != chess.WHITE:
                return
            self.selected_square = square
            self.legal_destinations = [m.to_square for m in self.board.legal_moves if m.from_square == square]
            self.draw_board()
        else:
            move = chess.Move(self.selected_square, square)
            if (self.board.piece_at(self.selected_square).piece_type == chess.PAWN and
                chess.square_rank(square) in (0, 7)):
                move = chess.Move(self.selected_square, square, promotion=chess.QUEEN)
            if move in self.board.legal_moves:
                self.make_human_move(move)
            else:
                piece = self.board.piece_at(square)
                if piece and piece.color == chess.WHITE:
                    self.selected_square = square
                    self.legal_destinations = [m.to_square for m in self.board.legal_moves if m.from_square == square]
                else:
                    self.selected_square = None
                    self.legal_destinations = []
                self.draw_board()

    def on_motion(self, event):
        pass

    # -------------------------
    # Moves / Game control
    # -------------------------
    def make_human_move(self, move: chess.Move):
        self.board.push(move)
        self.history.append(move)
        self.last_move = move
        self.selected_square = None
        self.legal_destinations = []
        self._append_move_to_list(move)
        self.draw_board()
        if self.board.is_game_over():
            self._show_game_over()
            return
        self._start_ai_move()

    def _append_move_to_list(self, move: chess.Move):
        ply = len(self.history)
        self.moves_text.configure(state=tk.NORMAL)
        if ply % 2 == 1:
            move_no = (ply + 1) // 2
            self.moves_text.insert(tk.END, f"{move_no:>2}. {move.uci():6} ")
        else:
            self.moves_text.insert(tk.END, f"{move.uci():6}\n")
        self.moves_text.see(tk.END)
        self.moves_text.configure(state=tk.DISABLED)

    def new_game(self):
        if self.ai_thinking:
            messagebox.showinfo("Please wait", "AI is thinking — try again shortly.")
            return
        self.board = chess.Board()
        self.history.clear()
        self.selected_square = None
        self.legal_destinations = []
        self.last_move = None
        self.moves_text.configure(state=tk.NORMAL)
        self.moves_text.delete("1.0", tk.END)
        self.moves_text.configure(state=tk.DISABLED)
        self.status_var.set("White to move")
        self.draw_board()

    def undo_move(self):
        if self.ai_thinking:
            messagebox.showinfo("Please wait", "AI is thinking — try again shortly.")
            return
        try:
            if len(self.history) >= 1:
                self.board.pop()
                self.history.pop()
            self._rebuild_move_list_from_history()
            self.selected_square = None
            self.legal_destinations = []
            self.last_move = None
            self.status_var.set("Your move")
            self.draw_board()
        except Exception as e:
            print("Undo error:", e)

    def _rebuild_move_list_from_history(self):
        self.moves_text.configure(state=tk.NORMAL)
        self.moves_text.delete("1.0", tk.END)
        for i, mv in enumerate(self.history):
            if i % 2 == 0:
                move_no = (i // 2) + 1
                self.moves_text.insert(tk.END, f"{move_no:>2}. {mv.uci():6} ")
            else:
                self.moves_text.insert(tk.END, f"{mv.uci():6}\n")
        self.moves_text.configure(state=tk.DISABLED)

    # -------------------------
    # AI threading / move handling
    # -------------------------
    def _start_ai_move(self):
        if self.board.is_game_over():
            return
        self.ai_thinking = True
        self.status_var.set(AI_THINK_TEXT)
        depth = max(1, int(self.ai_search_depth.get()))

        def target():
            start = time.time()
            move, val = minimax_root(self.board, depth, maximizing=False)
            elapsed = time.time() - start
            def after_move():
                if move is None:
                    self.ai_thinking = False
                    self.draw_board()
                    self._show_game_over()
                    return
                self.board.push(move)
                self.history.append(move)
                self.last_move = move
                self._append_move_to_list(move)
                self.ai_thinking = False
                self.status_var.set(f"Your move — AI eval {val} (took {elapsed:.2f}s)")
                self.draw_board()
                if self.board.is_game_over():
                    self._show_game_over()
            self.root.after(10, after_move)

        t = threading.Thread(target=target, daemon=True)
        t.start()
        self.ai_thread = t

    # -------------------------
    # Game over / dialogs
    # -------------------------
    def _show_game_over(self):
        result = "Game over.\n"
        if self.board.is_checkmate():
            winner = "Black (AI)" if self.board.turn == chess.WHITE else "White (You)"
            result += f"Checkmate — Winner: {winner}\n"
        elif self.board.is_stalemate():
            result += "Stalemate.\n"
        elif self.board.is_insufficient_material():
            result += "Draw — insufficient material.\n"
        else:
            result += "Draw or game over.\n"
        result += f"Result: {self.board.result()}"
        self.status_var.set(result)
        messagebox.showinfo("Game Over", result)

    # -------------------------
    # Clocks
    # -------------------------
    def _start_clock_thread(self):
        def clock_loop():
            last_tick = time.time()
            while self._running_clock:
                time.sleep(0.2)
                now = time.time()
                dt = now - last_tick
                last_tick = now
                if self.ai_thinking or not self.board.turn:
                    if not self.board.turn:
                        self.black_time += dt
                else:
                    if self.board.turn:
                        self.white_time += dt
                def upd():
                   
                    self.white_clock.config(text=f"You: {self._format_time(self.white_time)}")
                    self.black_clock.config(text=f"AI: {self._format_time(self.black_time)}")
                self.root.after(10, upd)
        t = threading.Thread(target=clock_loop, daemon=True)
        t.start()

: 